In [1]:
import os
import pandas as pd
import math
import numpy as np
from tqdm import tqdm, trange
from seqeval.metrics import classification_report,accuracy_score,f1_score

import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, \
                            get_linear_schedule_with_warmup

from sklearn import metrics
from ner_config import *
from ner_utils import *
from data_processor import i2b2Dataset, pad_batch
from model import Bert_BiLSTM_CRF

os.chdir('..')


In [2]:
train_data = pd.read_csv(data_path_train, sep="\t").astype(str)
dev_data = pd.read_csv(data_path_dev, sep="\t").astype(str)
test_data = pd.read_csv(data_path_test, sep="\t").astype(str)

In [3]:
train_dataset = i2b2Dataset(train_data)
train_iter = DataLoader(dataset=train_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=pad_batch,
                        pin_memory=True
                        )
dev_dataset = i2b2Dataset(dev_data)
dev_iter = DataLoader(dataset=dev_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=pad_batch,
                        pin_memory=True
                        )
test_dataset = i2b2Dataset(test_data)
test_iter = DataLoader(dataset=test_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=pad_batch,
                        pin_memory=True
                        )

In [4]:
best_model = None
_best_val_loss = float("inf")
_best_val_acc = -float("inf")


In [5]:
model = Bert_BiLSTM_CRF(tag2idx).to(DEVICE)
optimizer = Adam(model.parameters(), lr=LEARNING_RATE, eps=1e-6)
# Warmup
len_dataset = len(train_dataset)
total_steps = (len_dataset // BATCH_SIZE) * EPOCHS if len_dataset % BATCH_SIZE == 0 \
    else (len_dataset // BATCH_SIZE + 1) * EPOCHS
warm_up_ratio = 0.1
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warm_up_ratio * total_steps,
                                            num_training_steps=total_steps)


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
print('Train Start ...')
print("Num examples = %d"%(len(train_dataset)))
print("Batch size = %d"%(BATCH_SIZE))
print("Epochs = %d"%(EPOCHS))
for epoch in range(1, EPOCHS + 1):
    train(epoch, model, train_iter, optimizer, scheduler, DEVICE)
    if epoch % 5 == 0:
        print('valid-->', end='')
        candidate_model, loss, acc = validate(epoch, model, dev_iter, DEVICE)
        if loss < _best_val_loss and acc > _best_val_acc:
            best_model = candidate_model
            _best_val_loss = loss
            _best_val_acc = acc
y_test, y_pred = test(best_model, test_iter, DEVICE)

torch.save({'model': best_model.state_dict()}, os.path.join(NER_MODEL_SAVED_DIR, 'NER_bert_crf.ckpt'))
print('Train End ... Model saved')

Train Start ...
Num examples = 14387
Batch size = 32
Epoch: 1, Loss:33.2560
Epoch: 2, Loss:23.6891
Epoch: 3, Loss:17.4761
Epoch: 4, Loss:12.1402
Epoch: 5, Loss:8.4643
valid-->Epoch: 5, Val Loss:7.0093, Val Acc:86.421%
Epoch: 6, Loss:6.2904
Epoch: 7, Loss:5.1266
Epoch: 8, Loss:4.3912
Epoch: 9, Loss:3.8947
Epoch: 10, Loss:3.5283
valid-->Epoch: 10, Val Loss:3.1504, Val Acc:93.808%
Epoch: 11, Loss:3.2361
Epoch: 12, Loss:3.0039
Epoch: 13, Loss:2.8162
Epoch: 14, Loss:2.6704
Epoch: 15, Loss:2.5258
valid-->Epoch: 15, Val Loss:2.3592, Val Acc:95.121%
Epoch: 16, Loss:2.4335
Epoch: 17, Loss:2.3328
Epoch: 18, Loss:2.2405
Epoch: 19, Loss:2.1642
Epoch: 20, Loss:2.0957
valid-->Epoch: 20, Val Loss:2.0843, Val Acc:95.407%
Epoch: 21, Loss:2.0583
Epoch: 22, Loss:1.9757
Epoch: 23, Loss:1.9348
Epoch: 24, Loss:1.9172
Epoch: 25, Loss:1.8672
valid-->Epoch: 25, Val Loss:1.9988, Val Acc:95.571%
Epoch: 26, Loss:1.8364
Epoch: 27, Loss:1.7875
Epoch: 28, Loss:1.7631
Epoch: 29, Loss:1.7391
Epoch: 30, Loss:1.7000
val